<img src="https://raw.githubusercontent.com/RuneROe/git_color_by_similarity/master/logo.png" height="200" align="right" style="height:240px">

##SIMalign


**How to:**
1.   Install dependencies be running first cell (takes some minutes). The code will restart the kernel so wait for this to happen before proceeding.
2.   Run the "Importing files" cell.
3.   A bottum saying "Choose Files" will appear. Press it and choose all the structures that you want to analyse (At least 3 structures or use foldseek).
4.   Type which structure you want as your reference structure.
5.   Run the remaining cells.

In [ ]:
#@title Install dependencies

import os
def get_script(script):
    raw_script = f"https://raw.githubusercontent.com/RuneROe/git_color_by_similarity/master/{script}.py"
    local_script_path = f"/content/{script}.py"
    os.system(f"wget {raw_script} -O {local_script_path}")

scripts = {"findsurfaceatoms","foldseek_search","hotspot_finder","SIMalign","visualization"}
if not os.path.isfile("SIMalign_READY"):
    print("installing condacolab...")
    os.system("pip install -q condacolab")
    import condacolab
    condacolab.install()
    print("installing rdkit...")
    os.system("mamba install -c conda-forge rdkit")
    print("installing pymol...")
    os.system("mamba install -c conda-forge pymol-open-source")
    os.system("wget https://raw.githubusercontent.com/rdkit/rdkit/master/Docs/Book/data/cdk")
    os.system("which pymol")
    import subprocess
    cmd = subprocess.Popen(["pymol", "-cKRQ"])
    os.system("ps aux | grep pymol")
    os.system("pip install py3Dmol")    
    for s in scripts:
        get_script(s)
    os.system("pip install biopython")
    print("installing foldseek...")
    os.system("wget https://mmseqs.com/foldseek/foldseek-linux-avx2.tar.gz")
    os.system("tar xvzf foldseek-linux-avx2.tar.gz")
    os.system("export PATH=$(pwd)/foldseek/bin/:$PATH")
    os.system("touch SIMalign_READY")
    print("Please wait for the kernel to reboot before proceeding!")
else:
    print("Dependendies already installed.")

In [ ]:
#@title Importing files

#@markdown Import at least 3 files or activate foldseek.


from google.colab import files
import sys
import os



# Removing old uploads
OK_files = {"foldseek","foldseek_output",".config", "condacolab_install.log", "__pycache__", "SIMalign_READY", "ThermoDB_READY", "thermoDB", "foldseek_output", "sample_data","findsurfaceatoms.py","foldseek_search.py","hotspot_finder.py","SIMalign.py","visualization.py"}
for file in os.listdir():
    if file not in OK_files:
        os.remove(file)

# Wait until files are removed
while True:
    if set(os.listdir()).issubset(OK_files):
        break


# Checking if imported files are OK
infiles = files.upload()
infilenames = list(infiles.keys())

for file in infilenames:
    if " " in file:
        print("ERROR: No spaces in the names of the uploaded files are allowed.")
        sys.exit(1)
ref_structure = infilenames[0]


print("Choose a reference structure:")

# Prompt the user to choose a file
infile_set = set(infilenames)
while True:
    choice = input("Reference: ")
    number = 0
    for file in infilenames:
        if file.startswith(choice):
            number += 1
            ref_structure = file
    if number == 1:
        print(f'Selected reference structure: {ref_structure}')
        break
    elif number > 1:
        print("Not unique choice. Please enter full file name or remove files of similar names.")
    else:
        print("Invalid choice. Please enter a name of a file.")



In [ ]:
#@title Download options
download_pymol = True #@param {type: "boolean"}
#@markdown  - `download_pymol` allows you to download a pymol file of your aligned structures. 
outfile_name = "outfile" #@param {type:"string"}

download_alignment = True #@param {type: "boolean"}
#@markdown  - `download_alignment` allows you to download a alignment file of your aligned structures in sequence format.
alignment_file_name = "alignment.aln" #@param {type:"string"}


# removing spaces from outfile and add .pse
outfile_name = "_".join(outfile_name.split(" "))+".pse"

In [ ]:
#@title Visualization options
color_mode = "similarity" #@param ["similarity", "hotspot", "none"] {type:"string"}
#@markdown  - `color_mode` specify which way the structures should be colored.
structure_format = "spheres-sticks" #@param ["spheres-sticks","cartoon","spheres","sticks"] {type:"string"}
#@markdown   - `structure_format` specify how the structure should be showed in pymol.
show_in_pymol = "only_not_conserved" #@param ["only_not_conserved_core","only_core", "only_not_conserved","entire_chain_A","everything"] {type:"string"}
#@markdown   - `show_in_pymol` specify what part of the structures that will be shown in pymol.
color_by_element = True #@param {type: "boolean"}
#@markdown   - If `color_by_element` is ON then atom will be colored by element in pymol.

In [ ]:
#@title SIMalign options
max_iterations = 3 #@param {type:"integer"}
#@markdown  - `max_iterations` is the maximum number of alignments. A high number can lead to slow runtime.
min_aligned_aa = 100 #@param {type:"integer"}
#@markdown  - `min_aligned_aa` is how many amino acid that minimum should be used for alignment. A low number can lead to overfitting.
max_dist = 6 #@param {type:"integer"}
#@markdown  - `max_dist` is the maximum length between to amino acids before it is considered as a gab in the alignment. A too low number can lead to false gabs and a too high number can lead to false positive.
# remove_chain_duplicate = True #@param {type:"boolean"}
# For now it only takes chain A
#  #@markdown If `remove_chain_duplicate` is true then is chain duplicates removed from the structure.

In [ ]:
#@title Foldseek options
foldseek = True #@param {type:"boolean"}
#@markdown  - Activate foldseek by setting foldseek ON.
foldseek_database = "Alphafold/Swiss-Prot" #@param ["Alphafold/UniProt","Alphafold/UniProt50-minimal","Alphafold/UniProt50","Alphafold/Proteome","Alphafold/Swiss-Prot","ESMAtlas30","PDB","Thermophilic_DB"] {type:"string"}
#@markdown  - `foldseek_database` specify which database should be used for the foldseek search.
foldseek_variable_tresshold = "fident" #@param ["number_of_structures","evalue","pident","fident","nident","alnlen","bits","mismatch","qcov","tcov","lddt","qtmscore","ttmscore","alntmscore","rmsd","prob"]
#@markdown  - `foldseek_variable_tresshold` specify what foldseek variable that should be used as cutoff for structures. The defualt is "fident" which is sequence identity. "number_of_structures" enable the user to specify how many structures that should be downloaded.
foldseek_value_tresshold = 0.8  #@param {type:"number"}
#@markdown  - `foldseek_value_tresshold` specify the cutoff value for the given foldseek variable.
foldseek_search_against = "ref_structure" #@param ["ref_structure","all_structures"]
#@markdown  - `foldseek_search_against` specify wheather only the reference structure or all structures should be used in the foldseek search.
#@markdown Read more about foldseek on https://github.com/steineggerlab/foldseek

In [ ]:
#@title Hotspot finding options
find_hotspots = True #@param {type: "boolean"}
#@markdown  - If `find_hotspots` is true, then the program will find amino acid in the structure that can be mutated to potentially alter the stability of the protein.
#hotspot_min_size = 2 #@param {type: "integer"}
#For now we only finds single mutations

In [ ]:
#@title Run prediction
import SIMalign


if len(infilenames) < 3 and foldseek == False:
    print("ERROR: Import at least 3 files or activate foldseek.")
    sys.exit(1)

if foldseek:
    import foldseek_search
    infilenames = foldseek_search.run(foldseek_database,foldseek_variable_tresshold,foldseek_value_tresshold,foldseek_search_against,ref_structure,infilenames)
len_ref_structure, score_list, structure_list, core_selection = SIMalign.run(ref_structure, infilenames, max_iterations, min_aligned_aa, max_dist, alignment_file_name)

if find_hotspots:
    import hotspot_finder
    hotspot_list, exposed_list = hotspot_finder.run(structure_list,alignment_file_name)
    hotspot_finder.print_hotspot(hotspot_list,structure_list)
else:
    hotspot_list, exposed_list = None, None


if color_mode == "hotspot" and find_hotspots == False:
    print("ERROR: unable to color hotspot without finding them!")
else:
    import visualization
    visualization.run(color_mode,hotspot_list,score_list,structure_list,core_selection,exposed_list,structure_format,show_in_pymol,color_by_element)

from pymol import cmd
cmd.save(outfile_name)
if download_pymol:
    files.download(outfile_name)
else:
    print("Done")

In [ ]:
#@title Display reference structure
#@markdown Reference structure needs to be a pdb file in order to visualize.

import visualization
view = visualization.show_pdb(ref_structure,color_mode,score_list,len_ref_structure,hotspot_list)
view.zoomTo()